<a href="https://colab.research.google.com/github/shiernee/AI-Driven_Fetal_Monitoring_Model_to_Predict_FGR/blob/main/basic_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import math
import uuid
import sklearn
from sklearn.preprocessing import LabelEncoder
import warnings
import pandas as pd
import uuid
import logging
from logging.handlers import RotatingFileHandler

# Configure logging to write to a file
log_formatter = logging.Formatter('%(asctime)s [%(levelname)s] - %(message)s')
log_handler = logging.FileHandler('basic_cleaning_log.log')
log_handler.setFormatter(log_formatter)

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(log_handler)

In [ ]:
ante = pd.read_csv('/content/drive/MyDrive/SGA/Prev/fgr_df.csv')
ante = ante.rename(columns={
  "A-wave" : "a_wave",
  "AC" : "ac",
  "AF Index": "af_idx",
  "Amniotic fluid" : "af",
  "BPD" : 'bpd',
  "CM" : 'cm',
  "Case number" : 'case_no',
  "Cerebro-placental ratio" : 'cpr',
  "Cervix length" : 'cervix_len',
  "Cord" : 'cord',
  "Ductus Venosus PI" : "dv_pi",
  "EDF" : "edf",
  "EFW (clinical)" : "efw_clinical",
  "EFW centile" : 'efw_centile',
  "Estimated fetal weight" : 'efw',
  "Ethnic_group" : 'ethnic',
  "Exam" : 'exam',
  "Examination date" : 'exam_date',
  "FL": 'fl',
  "Fetal heart rate": 'fetal_heart_rate',
  "Fetus": 'fetus',
  "Funnelling" : 'funneling',
  "GA (DAYS)" : 'ga_days',
  "GA (WK)" : 'ga_week',
  "HC" : 'hc',
  "Hospital no." : 'hospital_no',
  "Humerus": "hl",
  "MCA PI (2)" : 'mca_pi',
  "MCA RI (2)" : 'mca_ri',
  "Notch" : 'notch',
  "PID" : 'pid',
  "Placenta site" : 'placenta_site',
  "Presentation" : 'presentation',
  "RI left" : 'ri_left',
  "RI right": 'ri_right',
  "TAV" : 'tav',
  "TCD" : 'tcd',
  "UA PI" : 'ua_pi',
  "UA RI" : 'ua_ri',
  "Uterine artery PI  left" :  'ua_pi_left',
  "Uterine artery PI  right" : 'ua_pi_right',
})
ante.info()

In [ ]:
def clean_hospital_no(df: pd.DataFrame):
  """
  Clean the hospital_no, and assign missing hospital_no with unique identifier
  Args:
    df (pd.DataFrame): original input DataFrame
  Return:
    df (pd.DataFrame): modified output DataFrame
  """
  try:
    # Drop entries with no hospital no
    n = df[df['hospital_no'].isna()]
    df = df.drop(n.index)

    # Filter hospital no that contains except 0 - 9 and .
    nint = df[df['hospital_no'].str.contains(r'[^0-9.]', na=False)]

    # Convert hospital no in int to str
    inth = df.drop(nint.index)
    inth['hospital_no'] = inth['hospital_no'].apply(lambda x: int(float(x)) if x.replace('.', '', 1).isdigit() else x).astype(str)

    # Setting constant UUID to same patient
    for id in n['pid'].unique():
        n.loc[n['pid'] == id, 'hospital_no'] = '~' + str(uuid.uuid4())

    # Merge all data
    df = pd.concat([n, nint, inth])

    # Optional: Assertion to check if 'hospital_no' column has no missing values
    assert not df['hospital_no'].isna().any(), "There are still missing values in the 'hospital_no' column."

    # Logging information
    logger.info("clean_hospital_no completed successfully.")

    return df

  except Exception as e:
    logger.error(f"Error at clean_hospital_no: {str(e)}")
    raise

In [ ]:
def restore_start_date(df: pd.DataFrame):
  """
  Correct and restore the start_date of pregnancy, also corrects the GA of the restored entries
  Args:
    df (pd.DataFrame): original input DataFrame
  Return:
    df (pd.DataFrame): modified output DataFrame
  """
  try:
    # Convert the total GA to days
    df['ga'] = df['ga_week'] * 7 + df['ga_days']
    # Calculate start date by subtracting examination date with gestational age
    df['ga'] = pd.to_numeric(df['ga'], errors='coerce')
    # Convert 'exam_date' to datetime if it's not already
    df['exam_date'] = pd.to_datetime(df['exam_date'])
    df['start_date'] = df['exam_date'] - pd.to_timedelta(df['ga'], unit='D')

    # Split the data into entreis with start_date and without
    withSD = df[df['start_date'].notna()]
    nonSD = df[df['start_date'].isna()]

    # Sort entries with hospital_no first, then with start_date
    withSD = withSD.sort_values(['hospital_no', 'start_date']).reset_index(drop=True)

    # Check current row hospital_no is same as next row hospital_no
    # [If same hospital_no then False, will not assign new ID]
    # Check current row pregnancy start_date is greater than next row pregnancy start_date
    # [If same hospital_no and current row pregnancy start_date is within 7 days of next row start_date, treat it as same patient]
    withSD['id'] = (withSD['hospital_no'] != withSD['hospital_no'].shift(1)) | (withSD['start_date'] > withSD['start_date'].shift(1) + pd.to_timedelta(7, unit='D'))
    withSD['id'] = np.cumsum(withSD['id'])

    # Group the data by the ID then select the mode of start_date occurence for that particular ID and to replace all other start date
    newSD = withSD.groupby('id')['start_date'].apply(lambda x: pd.Series.mode(x)[0]).to_dict()
    withSD['start_date'] = withSD['id'].apply(lambda x: newSD.get(x))

    # Assign id=0 to entries without start_date
    nonSD['id'] = 0
    nonSD['start_date'] = None

    # Get entries with start_date that has the same hospital_no with entries without start_date
    temp = withSD[withSD['hospital_no'].isin(nonSD['hospital_no'])]

    hosNo1 = list(temp['hospital_no'])
    startDate1 = list(temp['start_date'])
    id1 = list(temp['id'])

    hosNo2 = list(nonSD['hospital_no'])
    startDate2 = list(nonSD['start_date'])
    examDate2 = list(nonSD['exam_date'])
    id2 = list(nonSD['id'])

    # If entries without start_date have same hospital_no as entries with start_date
    # Get the days difference between exam_date (without start_date) and start_date (with start_date)
    # If days >= 0 and days <= 300, then assume they are the same patient, having same id and start_date
    # Assign start_date to entries with same hospital_no if their examination duration since start_date is not more than 300 days
    for i in range(len(hosNo2)):
      for j in range(len(hosNo1)):
        if hosNo2[i] == hosNo1[j]:
          days = examDate2[i] - startDate1[j]
          if days >= pd.to_timedelta(0, unit='D') and days <= pd.to_timedelta(300, unit='D'):
            startDate2[i] = startDate1[j]
            id2[i] = id1[j]

    # Assign start_date and ID
    nonSD['start_date'] = startDate2
    nonSD['id'] = id2

    # Get entries with recovered start_date
    recoveredSD = nonSD[nonSD['id'] != 0]
    nonSD = nonSD.drop(recoveredSD.index)

    # Get all entries with start_date
    # Dropped 15139 entries (140106 - 124967 = 15139)
    df = pd.concat([withSD, recoveredSD]).reset_index(drop=True)
    df = df.sort_values(['id', 'exam_date']).reset_index(drop=True)

    # Restore/correct GA
    df['ga'] = (df['exam_date'] - df['start_date']).dt.days

    assert df['start_date'].notna().all(), "Entries with NaN start_date should be corrected and restored."
    assert not df['ga'].isna().any(), "All entries should have a valid gestational age (ga)."
    assert (df['exam_date'] - df['start_date']).dt.days.equals(df['ga']), "Restored GA should be consistent with the corrected start_date."

    logger.info("restore_start_date completed successfully.")

  except Exception as e:
    logger.error(f"Error at restore_start_date: {str(e)}")
    raise

  return df

In [ ]:
def obtain_singleton(df: pd.DataFrame):
  try:
    # Assertion to check if 'id' and 'fetus' columns exist in the DataFrame
    assert 'id' in df.columns and 'fetus' in df.columns, "The 'id' or 'fetus' column is missing in the DataFrame."

    new_fetus = df.groupby(['id'])['fetus'].max()
    df['fetus_no'] = df["id"].apply(lambda x: new_fetus.get(x))
    df = df[df['fetus_no'] == 1.0]

    # Assertion to check if 'fetus_no' column has no missing values
    assert not df['fetus_no'].isna().any(), "There are still missing values in the 'fetus_no' column."

    # Logging information
    logger.info("obtain_singleton completed successfully.")

    return df

  except Exception as e:
    logger.error(f"Error at obtain_singleton: {str(e)}")
    raise

In [ ]:
def restore_bpd_ac_hc_fl(df):
  try:
    df['fl'] = np.where(df['fl'].isna(), df['ac'] * 0.22, df['fl'])
    df['bpd'] = np.where(df['bpd'].isna(), df['hc'] * 78 / (89 * math.pi), df['bpd'])
    df['fl'] = np.where(df['fl'].isna(), df['bpd'] * 0.79, df['fl'])

    df['ac'] = np.where(df['ac'].isna(), df['fl'] / 0.22, df['ac'])
    df['bpd'] = np.where(df['bpd'].isna(), df['fl'] / 0.79, df['bpd'])
    df['hc'] = np.where(df['hc'].isna(), df['bpd'] * (89 * math.pi) / 78, df['hc'])

    logger.info("restore_bpd_ac_hc_fl completed successfully.")
    return df

  except Exception as e:
    logger.error(f"Error at restore_bpd_ac_hc_fl: {str(e)}")
    raise

In [ ]:
def compute_EFW(df):
  """
  Compute Estimated Fetal Weight (EFW)
  Args:
    df (pd.DataFrame): original input DataFrame
  Return:
    df (pd.DataFrame): modified output DataFrame
  """
  # Formula to compute EFW
  # If BPD, AC, FL and HC is not NA, then calculate EFW using these values
  # Else use the predefined EFW
  try:
    df['efw'] = np.where(
      df[['bpd', 'ac', 'fl', 'hc']].notna().all(1),
      round(10 ** (1.3596 +
      0.0064 * df['hc']/10 +
      0.0424 * df['ac']/10 +
      0.174 * df['fl']/10 +
      0.00061 * df['bpd']/10 * df['ac']/10 -
      0.00386 * df['ac']/10 * df['fl']/10)),
      np.where(
        isinstance(df['efw'], float) or isinstance(df['efw'], int),
        df['efw'],
        np.nan
      )
    )
    logger.info("compute_EFW completed successfully.")
    return df

  except Exception as e:
    logger.error(f"Error at compute_EFW: {str(e)}")
    raise

In [ ]:
def compute_CPR(df):
  """
  Compute Cerebellar-Placenta Ratio
  Args:
    df (pd.DataFrame): original input DataFrame
  Return:
    df (pd.DataFrame): modified output DataFrame
  """
  # Compute CPR
  df['cpr'] = np.where(df[['mca_pi', 'ua_pi']].notna().all(1), df['mca_pi']/df['ua_pi'], df['cpr'])
  logger.info("compute_CPR completed successfully.")

  return df

In [ ]:
# https://www.ncbi.nlm.nih.gov/books/NBK441881/
# A normal amniotic fluid index is 5 cm to 25 cm using the standard assessment method. Less than 5 cm is considered oligohydramnios, and greater than 25  cm is considered polyhydramnios.

def compute_AFI(df):
  df['af'] = np.where(df['af_idx'].notna(),
    pd.cut(df['af_idx'], bins = [0, 5, 25, np.inf], labels = ['oligohydramnios', 'normal', 'polyhydramnios']),
    df['af'])
  df.replace({'anhydramnios' : 'oligohydramnios', 'reduced' : 'oligohydramnios', 'increased' : 'polyhydramnios'}, inplace = True)

  logger.info("compute_AFI completed successfully.")

  return df

In [ ]:
def compute_EFW_centile(df, centile_df):
  try:
    centile_df = centile_df.rename(columns={'GA' : 'ga'})
    centile_df['ga'] = centile_df['ga'].astype(int)

    df = pd.merge(df, centile_df, on = 'ga', how = 'left')
    refCentile = list(centile_df.columns)[1:]
    ranges = df[[2.5, 5, 10, 25, 50, 75, 90, 95, 97.5]].values.tolist()
    efw = df['efw'].values.tolist()
    centile = []

    for i in range(len(efw)):
      if efw[i] >= ranges[i][0] and efw[i] <= ranges[i][-1]:
        for j in range(len(ranges[i])):
          if efw[i] == ranges[i][j]:
            centile.append(refCentile[j])
            break
          if efw[i] < ranges[i][j]:
            centile.append(refCentile[j] - (refCentile[j] - refCentile[j - 1]) * (ranges[i][j] - efw[i]) / (ranges[i][j] - ranges[i][j - 1]))
            break
      else:
        centile.append(0)

    df['efw_centile'] = centile
    df.drop(df[df['efw_centile'] == 0].index, inplace = True)
    df['cur_sga'] = df['efw_centile'] <= 10

    assert not df['efw_centile'].isna().any(), "There are null values in the 'efw_centile' column after restoration."
    logger.info("compute_EFW_centile completed successfully.")
    return df

  except Exception as e:
    logger.error(f"Error at compute_EFW_centile: {str(e)}")
    raise



In [ ]:
def clean_antenatal_data(df):
  """
  Clean up antenatal data
  Args:
    df (pd.DataFrame): original input DataFrame
  Return:
    df (pd.DataFrame): cleaned output DataFrame
  """
  # Steps to clean antenatal data
  df = clean_hospital_no(df)
  df = restore_start_date(df)
  df = obtain_singleton(df)
  df = restore_bpd_ac_hc_fl(df)
  df = compute_EFW(df)
  df = compute_CPR(df)
  df = compute_AFI(df)
  centile_df = pd.read_excel('/content/drive/MyDrive/SGA/Prev/EFW centile.xlsx')
  df = compute_EFW_centile(df, centile_df)

  return df

In [ ]:
post = pd.read_csv('/content/drive/MyDrive/SGA/Years/IUGR_Studies.csv')
post = post.rename(columns={
  'Hospital No' : 'hospital_no', # Cleaned
  'Fetus No' : 'fetus_no_birth', # No need to clean since all same value of 1
  "Mother's Date of Birth (dd/mm/yy)" : "mother_dob",
  "Mother's Age at Delivery (Yrs)" : "mother_age",
  "Mother weight taken Date (dd/mm/yy)" : "mother_weight_taken_date",
  "Mother Weight (kg) at first visit" : "mother_weight",
  "Mother Height (cm)" : "mother_height",
  "Mother Smoking (Yes = 1, No = 0]" : "smoking",
  "Hypertension - Pregancy Induced or Essential [Nil = 0, PIH = 1, Essential HpT = 2]" : "hypertension",
  "Diabetes - Gestational or Pregestational                                     [Nil=0, GDM=1, PRE-GDM=2]": "diabetes",
  "Others (State)" : "other_disease",
  'Date of Delivery (dd/mm/yy)' : "date_of_delivery",
  'Mode of Delivery [SVD=1, Forceps=2, Vacuum=3, LSCS=4,D&C=5, Breech delivery=6]': "mode_of_delivery",
  'GA at birth (week)' : "ga_birth_week",
  'GA at birth (day)' : "ga_birth_day",
  'GA at birth (completed weeks + days = convert to decimal point, e.g. 24 weeks + 5 days "= 24 +5/7" = 24.71)': 'ga_birth',
  'Baby Gender (F/M) (A=Ambigious)': 'gender',
  'Birthweight (g) ' : 'birth_weight',
  'Birth Length (cm)' : "birth_length",
  'Head circumference (cm)' : 'hc_birth',
  'Apgar Score (1min)' : 'apgar_1_min',
  'Apgar Score (5min)' : 'apgar_5_min',
  'Neonatal Outcome (Alive = 1/ Dead = 2)': 'neonatal',
  'Admission to NICU/ PICU/ WARD [NICU/SCN=1, PICU=2, Ward=3, Maternity (Not admitted)=4,Mortuary=5]' : 'admission',
  'Death Diagnosis (no death = 0,  yes = 1)' : 'death'
})
post.info()

In [ ]:
def clean_postnatal_data(df):
  try:
    # Remove weird data
    df.replace(['1-Jan-1290', '20-Apr-20'], np.nan, inplace = True)
    # Convert Hospital no. to str
    df['hospital_no'] = df['hospital_no'].astype(str)
    # Convert the total GA to days
    df['ga_birth'] = df['ga_birth_week'] * 7 + df['ga_birth_day']
    df['ga_birth'] = pd.to_numeric(df['ga_birth'], errors='coerce')
    # Convert 'date_of_delivery' to datetime if it's not already
    df['date_of_delivery'] = pd.to_datetime(df['date_of_delivery'])
    # Calculate Start Date from Delivery Date
    df['start_date_post'] = df['date_of_delivery'] - pd.to_timedelta(df['ga_birth'], unit = 'D')
    # Remove dead fetus
    df = df[(df['neonatal'] == 1) & (df['death'] == 0)]

    assert not any(df['date_of_delivery'].isna()), "There are missing values in the 'date_of_delivery' column after cleaning."
    assert not any(df['ga_birth'].isna()), "There are missing values in the 'ga_birth' column after cleaning."
    assert not any(df['start_date_post'].isna()), "There are missing values in the 'start_date_post' column after cleaning."

    logger.info("compute_EFW_centile completed successfully.")
    return df

  except Exception as e:
    logger.error(f"Error at clean_postnatal_data: {str(e)}")
    raise


In [ ]:
def merge_clean_ante_post(ante, post):
  try:
    df = pd.merge(ante, post, on = ['hospital_no'], how = 'left')
    df = df[(df['start_date'] >= df['start_date_post'] - pd.DateOffset(days = 7)) & (df['start_date'] <= df['start_date_post'] + pd.DateOffset(days = 7))]
    df['gender'].replace({'B' : 'M'}, inplace = True)
    df['start_date'] = pd.to_datetime(df['start_date'])
    df['mother_dob'] = pd.to_datetime(df['mother_dob'])

    # Calculate 'mother_age_at_start_date'
    df['mother_age_at_start_date'] = (
      (df['start_date'].dt.month == df['mother_dob'].dt.month) &
      (df['start_date'].dt.day < df['mother_dob'].dt.day)
    ) | (df['start_date'].dt.month < df['mother_dob'].dt.month)
    df['mother_age_at_start_date'] = df['start_date'].dt.year - df['mother_dob'].dt.year - df['mother_age_at_start_date']

    logger.info("merge_clean_ante_post completed successfully.")
    return df

  except Exception as e:
    logger.error(f"Error at merge_clean_ante_post: {str(e)}")
    raise


In [ ]:
def merge_groundtruth(df):
  gt = pd.read_csv('/content/drive/MyDrive/SGA/Ref_Centile/I21_BW.csv')
  gt.columns = gt.columns.str.lower()
  gt.rename(columns = {'ga' : 'ga_birth'}, inplace = True)
  df = pd.merge(df, gt, on = ['ga_birth', 'gender'], how = 'left')
  df['sga'] = df['birth_weight'] / 1000 <= df['p_10']
  df['lbw'] = df['birth_weight'] / 1000 <= 2.5
  df['status_change'] = df['cur_sga'] ^ df['sga']

  return df

In [ ]:
cleaned_ante = clean_antenatal_data(ante)
cleaned_post = clean_postnatal_data(post)
df = merge_clean_ante_post(cleaned_ante, cleaned_post)
df = merge_groundtruth(df)

In [ ]:
# Obtain neccessary cell and label encoding
df = df[['a_wave', 'ac', 'af', 'bpd', 'cm', 'cpr', 'cervix_len', 'cord', 'dv_pi', 'edf', 'efw_centile', 'efw', 'fl', 'fetal_heart_rate', 'funneling', 'ga', 'hc', 'hl', 'id', 'mca_pi', 'mca_ri', 'notch', 'placenta_site', 'presentation', 'tcd', 'ua_pi', 'ua_ri', 'ua_pi_left', 'ua_pi_right', 'mother_age_at_start_date', 'mother_height', 'mother_weight', 'smoking', 'hypertension', 'diabetes', 'gender', 'sga', 'lbw', 'cur_sga', 'status_change', 'birth_weight']]
#df.info()

In [ ]:
tri2 = df[df['ga'] < 182]
tri3 = df[df['ga'] >= 182]
tri2.dropna(thresh = len(tri2) * 0.4, axis = 1, inplace = True)
tri3.dropna(thresh = len(tri3) * 0.4, axis = 1, inplace = True)
tri2.info()
tri3.info()